# Machine Learning Fundamentals and Regularization

# Objectives

- Explain the notion of "validation data"
- Use the algorithm of cross-validation (with `sklearn`)
- Explain the concept of regularization
- Use Lasso and Ridge regularization in model design

# When a Good Model Goes Bad

One of the goals of a machine learning project is to make models which are highly predictive.
If the model fails to generalize to unseen data then the model is bad.

Adding complexity to a model can find patterns to help make better predictions! 

But too much complexity can lead to the model finding patterns in the noise...

<img src="images/overfitting_model_meme.jpeg" alt="complex model conspiracy theory meme, image from It's Always Sunny in Philadelphia" width=600>

>So how do we know when our model is ~~a conspiracy theorist~~ overfitting?

## Bias-Variance Tradeoff

1. High bias - **UNDERFITTING**
    1. Systematic error in predictions
    2. Bias is about the strength of assumptions the model makes
    3. Underfit models tend to have high bias
2. High variance - **OVERFITTING**
    1. The model is highly sensitive to changes in the data
    2. Overfit models tend to have low bias

<img src="images/bias_vs_variance.png" width=650>

##### Aside: Example of high bias and variance

High bias is easy to wrap one's mind around: Imagine pulling three red balls from an bucket that has hundreds of balls of all colors in a uniform distribution. Then my sample is a terrible representative of the whole population. If I were to build a model by extrapolating from my sample, that model would predict that _every_ ball produced would be red! That is, this model would be incredibly biased.

High variance is a little bit harder to visualize, but it's basically the "opposite" of this. Imagine that the population of balls in the bucket is mostly red, but also that there are a few balls of other colors floating around. Now imagine that our sample comprises a few balls, none of which is red. In this case, we've essentially picked up on the "noise", rather than the "signal". If I were to build a model by extrapolating from my sample, that model would be needlessly complex. It might predict that balls drawn before noon will be orange and that balls drawn after 8pm will be green, when the reality is that a simple model that predicted 'red' for all balls would be a superior model!

The important idea here is that there is a *trade-off*: If we have too few data in our sample (training set), or too few predictors, we run the risk of high *bias*, i.e. an underfit model. On the other hand, if we have too many predictors (especially ones that are collinear), we run the risk of high *variance*, i.e. an overfit model.

[Here](https://en.wikipedia.org/wiki/Overfitting#/media/File:Overfitting.svg) is a nice illustration of the difficulty.

# How Do We Identify a Bad Model? 🕵️

### Answer: More Complicated Model Validation

Generally speaking we want to take more precautions than using just a test and train split. After all, we're still imagining building just one model on the training set and then crossing our fingers for its performance on the test set.

Data scientists often distinguish *three* subsets of data: **training, validation, and testing**


Roughly:
- Training data is for building the model
- Validation data is for *tweaking* the model
- Testing data is for evaluating the model on unseen data

- Think of **training** data as what you study for a test
- Think of **validation** data is using a practice test (sometimes called the development or dev set)
- Think of **testing** data as the actual test!
    - A **holdout** set is when your test dataset is never used for training (unlike in cross-validation)


<img src="https://scikit-learn.org/stable/_images/grid_search_workflow.png" width=600>

> Image from Scikit-Learn: https://scikit-learn.org/stable/modules/cross_validation.html

### Steps:

1. Split data into training data and a holdout test
2. Design a model
3. Evaluate how well it generalizes with **cross-validation** (only training data)
4. Determine if we should adjust model, use cross-validation to evaluate, and repeat
    - 'Adjusting' model here means things like changing what features, building/engineering new features, but also adjusting _hyperparameters_
5. After iteratively adjusting your model, do a _final_ evaluation with the holdout test set
6. THEN DON'T TOUCH THE MODEL!!!

**Question**: What's different about this procedure from what we've described before? Aren't I just calling the test data "validation data" now? Is there any substantive difference?

- 


### From Validation to Cross-Validation

Since my model will "see" the validation data in any case, I might as well use *all* of my training data to validate my model! How do I do this?

Cross-validation works like this: First I'll partition my training data into $k$-many *folds*. Then I'll train a model on $k-1$ of those folds and "test" it on the remaining fold. I'll do this for all possible divisions of my $k$ folds into $k-1$ training folds and a single "testing" fold. Since there are $k\choose 1$$=k$-many ways of doing this, I'll be building $k$-many models!

![](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

##### Python Example

In [ ]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
birds = sns.load_dataset('penguins')
# Renaming columns in case anyone has the old version of the dataset with 'culmen'
birds.columns = ['species', 'island', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'sex']
birds.sample(5)

In [ ]:
birds.info()

In [ ]:
# For simplicity's sake we'll limit our analysis to the numeric columns.
df = birds[['bill_length_mm', 'bill_depth_mm',
                 'flipper_length_mm', 'body_mass_g']]

In [ ]:
# We'll drop the rows with null values
df = df.dropna().reset_index()

Suppose I want to model `body_mass_g` as a function of the other attributes.

In [ ]:
# Create our X and y
X = None
y = None

We'll make ten models and record our evaluations of them.

In [ ]:
# Instantiate (but no need to fit yet!) a Linear Regression model


In [ ]:
# Use cross_validate to model 10 different times, across 10 different folds
# Explore the arguments - might set different scoring, or return train scores
cv_results = None

In [ ]:
# Explore our results
cv_results.keys()

In [ ]:
print(f"Avg Train R2: {cv_results['train_r2'].mean():.4f} +/- {cv_results['train_r2'].std():.4f}")

In [ ]:
print(f"Avg Validation R2: {cv_results['test_r2'].mean():.4f} +/- {cv_results['test_r2'].std():.4f}")

#### Evaluate:

- 


But now what? How can we fix that?

# Preventing Overfitting - Regularization

Again, complex models are very flexible in the patterns that they can model but this also means that they can easily find patterns that are simply statistical flukes of one particular dataset rather than patterns reflective of the underlying process or trend we're trying to model.

When a model has large weights (coefficients), the model is "too confident". This translates to a model with high variance - which puts it in danger of overfitting!

We can combat overfitting/high variance with regularization.

### Some Types of Regularization:

1. Reducing the number of features
2. Increasing the amount of data
3. Penalizing coefficients: Ridge, Lasso, Elastic Net
        

## The Strategy Behind Ridge / LASSO / Elastic Net

Regularization is about introducing a factor into our model designed to enforce the stricture that the coefficients stay small, by _penalizing_ the ones that get too large.

Ridge and LASSO (or the combination of the two, called Elastic Net) regression are examples of **penalized** estimation. Penalized estimation makes some or all of the coefficients smaller in magnitude (closer to zero). Some of the penalties have the property of performing both variable selection (setting some coefficients exactly equal to zero) and shrinking the other coefficients. 


### LASSO (Least Absolute Shrinkage and Selection Operator): L1 Regularization - Absolute Value

- Tend to get sparse vectors (small weights go to 0)
- Reduce number of weights
- Good feature selection to pick out importance

<details>$$ \text{LASSO Cost Function} = -\dfrac{1}{m} \sum^m_{i=1}\big[\mathcal{L}(\hat y_i, y_i)+ \lambda|w_i| \big]$$
</details>

### Ridge: L2 Regularization - Squared Value

- Not sparse vectors (weights homogeneous & small)
- Tends to give better results for training
    
<details>$$ \text{Ridge Cost Function} = -\dfrac{1}{m} \sum^m_{i=1}\big[\mathcal{L}(\hat y_i, y_i)+ \lambda w_i^2 \big]$$
</details>

### Standardization before Regularization!

An important step before using either Lasso or Ridge regularization is to first standardize your data such that it is all on the same scale. Regularization is based on the concept of penalizing larger coefficients, ***so if you have features that are on different scales, some will get unfairly penalized***. A downside of standardization is that the value of the coefficients become less interpretable and must be transformed back to their original scale if you want to interpret how a one unit change in a feature impacts the target variable.

### 🤔 Which Do I Use?

### Ridge:
* We can "shrink down" prediction variables effects instead of deleting/zeroing them
* When you have features with high multicollinearity, the coefficients are automatically spread across them (you won't have redundancy)
* Since includes all features it can be computationally expensive (for many variables)

### Lasso:
* When you have a lot of variables it performs feature selection for you!
* Multicollinearity is also dealt with

### But in general: L2 

- Ridge makes for a gentler reining in of runaway coefficients. When in doubt, try Ridge first.
- LASSO will more quickly reduce the contribution of individual predictors down to insignificance. It is therefore most useful for trimming through the fat of datasets with many predictors or if a model with very few predictors is especially desirable.

### The Best of Both Worlds: Elastic Net

There is a combination of L1 and L2 regularization called the Elastic Net that can also be used. The idea is to use a scaled linear combination of both LASSO and Ridge, where the weights add up to 100%. We might want 50% of each, but we also might want, say, 10% Lasso and 90% Ridge.

## Code it Out!

### Producing a Very Overfit Model

We can often produce an overfit model by including lots of extra features, like with adding programmatically-generated **interaction terms**. We'll start over with the penguins dataset. This time we'll include the categorical features.

#### Train-Test Split

In [ ]:
birds = birds.dropna()
birds.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    birds.drop('body_mass_g', axis=1),
    birds['body_mass_g'],
    random_state=42
)

In [ ]:
# Taking in other features (category)
# Note that this method is equivalent to using a column transformer
ohe = OneHotEncoder(drop='first')
dummies = ohe.fit_transform(X_train[['species', 'island', 'sex']])

# Getting a DF
dummies_df = pd.DataFrame(dummies.todense(), columns=ohe.get_feature_names(),
                         index=X_train.index)

# What we'll feed int our model
X_train_df = pd.concat([X_train[['bill_length_mm', 'bill_depth_mm',
                                'flipper_length_mm']], dummies_df], axis=1)
X_train_df.head()

Our Test Data:

In [ ]:
# Note the same transformation (not FIT) to match structure
test_dummies = ohe.transform(X_test[['species', 'island', 'sex']])
test_df = pd.DataFrame(test_dummies.todense(), columns=ohe.get_feature_names(),
                       index=X_test.index)
X_test_df = pd.concat([X_test[['bill_length_mm', 'bill_depth_mm',
                              'flipper_length_mm']], test_df], axis=1)

#### Add Polynomial Features

In [ ]:
# Instantiate our polynomial feature creator
pf = PolynomialFeatures(degree=3)

X_poly_train = pf.fit_transform(X_train_df)
X_poly_test = pf.transform(X_test_df)

Train the model and evaluate (with cross-validation)

In [ ]:
# Instantiate a linear regression model
poly_lr = None

In [ ]:
# Now cross validate that model on X_poly_train and y_train
cv_results = None

In [ ]:
print(f"Avg Train R2: {cv_results['train_r2'].mean():.4f} +/- {cv_results['train_r2'].std():.4f}")
print(f"Avg Validation R2: {cv_results['test_r2'].mean():.4f} +/- {cv_results['test_r2'].std():.4f}")

##### Peeking at the end (test data) 👀

In [ ]:
poly_lr.fit(X_poly_train, y_train)
poly_lr.score(X_poly_test, y_test) # Eeek

### Ridge (L2) Regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

In [ ]:
# ALWAYS scale before regularized regression

ss = StandardScaler()

X_train_processed = ss.fit_transform(X_poly_train)
X_test_processed = ss.transform(X_poly_test)

In [ ]:
# Now instantiate our Ridge model!
# Note that it needs arguments to initialize
# Let's discuss arguments - what should we use as our alpha ?

rr = None

In [ ]:
# Now cross validate
cv_results = None

In [ ]:
print(f"Avg Train R2: {cv_results['train_r2'].mean():.4f} +/- {cv_results['train_r2'].std():.4f}")

print(f"Avg Validation R2: {cv_results['test_r2'].mean():.4f} +/- {cv_results['test_r2'].std():.4f}")

**Peeking ahead (test data) 👀**

In [ ]:
rr.fit(X_train_processed, y_train)
rr.score(X_test_processed, y_test)

#### Evaluate:

- 


### Optimizing the Regularization Hyperparameter - aka what to set as 'alpha'

The regularization strength could sensibly be any nonnegative number, so there's no way to check "all possible" values. It's often useful to try several values that are different orders of magnitude.

In [ ]:
alphas = [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000, 10_000]
train_scores = []
test_scores = []

for alpha in alphas:
    rr = Ridge(alpha=alpha, random_state=42)
    rr.fit(X_train_processed, y_train)
    train_score = rr.score(X_train_processed, y_train)
    test_score = rr.score(X_test_processed, y_test)
    
    train_scores.append(train_score)
    test_scores.append(test_score)

In [ ]:
fig, ax = plt.subplots()
plt.xscale('log')
plt.title('Ridge $R^2$ as a function of regularization strength')
ax.set_xlabel('Regularization strength $\lambda$')
ax.set_ylabel('$R^2$')
ax.plot(alphas, train_scores, label='train')
ax.plot(alphas, test_scores, label='test')
plt.legend();

#### Observation

Notice how the values increase but then decrease? Regularization helps with overfitting, but if the strength of the regularization becomes too great, then large coefficients will be punished more than they really should. What happens then is that the original error between truth and model predictions becomes neglected as a quantity to be minimized, and the bias of the model begins to outweigh its variance.

It looks like the best value is somewhere around 100. If we wanted more precision, we could repeat the same sort of exercise with a set of alphas nearer to 100.

NOTE: We'll keep learning better tools to adjust these kinds of _hyperparameters_!

## Your Turn!

Run two models: both a Ridge and a LASSO regresssion model (without cross-validation) on the processed X data. Then, explore the coefficients. What do you notice?

You can use the default `alpha` parameter for both models. Use `random_state=42`

In [ ]:
# Instantiate your Ridge regression

# Fit

# Score on train

# Score on test

In [ ]:
# Instantiate your LASSO regression

# Fit

# Score on train

# Score on test

In [ ]:
# Explore LASSO coefficients

In [ ]:
# Explore Ridge coefficients

#### Evaluate: What do you notice? Any observations?

- 


----

## LEVEL UP - Elastic Net!

Naturally, the Elastic Net has the same interface through sklearn as the other regularization tools! The only difference is that we now have to specify how much of each regularization term we want. The name of the parameter for this (represented by $\rho$ above) in sklearn is `l1_ratio`.

In [ ]:
enet = ElasticNet(alpha=10, l1_ratio=0.1, random_state=42)

enet.fit(X_train_processed, y_train)

In [ ]:
enet.score(X_train_processed, y_train)

In [ ]:
enet.score(X_test_processed, y_test)

Setting the `l1_ratio` to 1 is equivalent to the lasso:

In [ ]:
ratios = np.linspace(0.01, 1, 100)

In [ ]:
preds = []
for ratio in ratios:
    enet = ElasticNet(alpha=100, l1_ratio=ratio, random_state=42)
    enet.fit(X_train_processed, y_train)
    preds.append(enet.predict(X_test_processed[0].reshape(1, -1)))

In [ ]:
fig, ax = plt.subplots()

lasso = Lasso(alpha=100, random_state=42)
lasso.fit(X_train_processed, y_train)
lasso_pred = lasso.predict(X_test_processed[0].reshape(1, -1))

ax.plot(ratios, preds, label='elastic net')
ax.scatter(1, lasso_pred, c='k', s=70, label='lasso')
plt.legend();

#### Note on `ElasticNet()`

Is an Elastic Net with `l1_ratio` set to 0 equivalent to the ridge? In theory yes. But in practice no. It looks like the `ElasticNet()` predictions on the first test data point as `l1_ratio` shrinks are tending toward some value around 3400. Let's check to see what prediction `Ridge()` gives us:

In [ ]:
ridge = Ridge(alpha=10, random_state=42)
ridge.fit(X_train_processed, y_train)
ridge.predict(X_test_processed[0].reshape(1, -1))[0]

If you check the docstring for the `ElasticNet()` class you will see:
- that the function being minimized is slightly different from what we saw above; and
- that the results are unreliable when `l1_ratio` $\leq 0.01$.

**Exercise**: Visualize the difference in this case between `ElasticNet(l1_ratio=0.01)` and `Ridge()` by making a scatterplot of each model's predicted values for the first ten points in `X_test_processed`. Use `alpha=10` for each model.

        Level Up: Make a second scatterplot that compares the predictions on the same data
        points between ElasticNet(l1_ratio=1) and Lasso().

<details>
    <summary> Answer
    </summary>
    <code>fig, ax = plt.subplots()
enet_r = ElasticNet(alpha=10, l1_ratio=0.01, random_state=42)
enet_r.fit(X_train_processed, y_train)
preds_enr = enet_r.predict(X_test_processed[:10])
preds_ridge = ridge.predict(X_test_processed[:10])
ax.scatter(np.arange(10), preds_enr)
ax.scatter(np.arange(10), preds_ridge);</code>  
        </details>

<details>
    <summary>
        Level Up
    </summary>
<code>fig, ax = plt.subplots()
enet_l = ElasticNet(alpha=10, l1_ratio=1, random_state=42)
enet_l.fit(X_train_processed, y_train)
preds_enl = enet_l.predict(X_test_processed[:10])
preds_lasso = lasso.predict(X_test_processed[:10])
ax.scatter(np.arange(10), preds_enl)
ax.scatter(np.arange(10), preds_lasso);</code>
    </details

#### Fitting Regularized Models with Cross-Validation

Our friend `sklearn` also includes tools that fit regularized regressions *with cross-validation*: `LassoCV`, `RidgeCV`, and `ElasticNetCV`.

**Exercise**: Use `RidgeCV` to fit a seven-fold cross-validated ridge regression model to our `X_train_processed` data and then calculate $R^2$ and the RMSE (root-mean-squared error) on our test set.

<details>
    <summary>
        Answer
    </summary>
    <code>rcv = RidgeCV(cv=7)
rcv.fit(X_train_processed, y_train)
rcv.score(X_test_processed, y_test)
np.sqrt(mean_squared_error(y_test, rcv.predict(X_test_processed)))</code>
    </details>